
# RIVER OVERFLOW IN MARESME

# 1 - Introduction

The objective of this test is to evaluate the candidate’s ability working with Python and Geospatial data. You are asked to build a Python Notebook to assess the impact of floodings during the Gloria storm (**Januray 15th-25th, 2020**) in an area of interest of your choosing inside the Maresme county along the Tordera river (Catalonia, Spain).

You are asked to produce at least one of the following analytics:

* Number of square kilometers affected by the floodings.

* The affected population.

* The affected roads.

You must use satellite imagery in your analysis (consider using free Sentinel data) for point 1 and combine the results with open data for points 2 and 3. Aspects such as code readability, appealing visualizations and reproducibility will be highly valued.

In [ ]:
# MODULE                                             # DESCRIPTION
import numpy as np                                   # scientific computing
import numpy.ma as ma                                # numpy masked arrays
import pandas as pd                                  # data analysis and manipulation
import geopandas as gpd                              # geospatial data analysis
import folium                                        # interactive data visualization
import re                                            # regular expressions
import matplotlib.pyplot as plt                      # create visualizations
import seaborn as sns                                # create visualizations
import datetime                                      # datetime manipulation
import glob                                          # unix pathname expansion
import rioxarray                                     # rasterio xarray extension
import xarray as xr
from pathlib import Path                             # 
import math                                          #
import random     
from os import listdir
from os.path import isfile, join


def S2_getDate(filename) :
    basename = Path(filename).stem  
    try :
        found = re.search('S2(A|B)2A_(\d+)_.*',basename).group(2)
        dt = datetime.datetime.strptime(found, '%Y%m%d')
        
    except AttributeError:
        raise ValueError('Error: Date can not be extracted from filename %s .' % filename)
        
    return dt

The Tordera river extends over the Maresme county in Cataluña, Spain. In the following image the Tordera river basin is depicted:

![](./img/tordera.png)

The river flows into the Mediterranean Sea in Malgrat del Mar.

In [ ]:
tordera = gpd.read_file('./sentinel2/tordera_basin.geojson')

m = folium.Map(location = [41.732289, 2.661523],
               zoom_start = 11,
               tiles = "CartoDB positron")


for _, r in tordera.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m)
m

# 2 - Satellite data

First of all we need the satellite data, concretely we will use the Sentinel-2(S2) satellite imagery, whic is well suited to monitorize this events due to its high resolution of 10ms/px. S2 updates new information for a given place each 5 days.

Tho download the images we will use the [sen2r](https://sen2r.ranghetti.info/) library, the required images are already stored in the `/sentinel`directory. The following scripts execute the `runSentinel.sh` script for the range `[2019-01-01 - 2020-02-15]`. This code already downloads the corrected [level 2A](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a) images and crop its to the required shape.

Also the script computes the following products: 

- [Scene Classification Layer - SCL](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm)
- Bottom of Atmosphere (BOA) Reflectance

In [ ]:
# Run inside ./sentinel folder: 
# ./runSentinel.sh ./inputs-config.json tordera_basin.geojson 'tordera' '2019-12-01' '2020-02-01'

In [ ]:
! ls -l ./sentinel2/out/SCL

In [ ]:
scl_colors = ["black",       # 0 - NO_DATA
              "red",         # 1 - SATURATED_OR_DEFECTIVE
              "dimgrey",     # 2 - DARK_AREA_PIXELS
              "saddlebrown", # 3 - CLOUD_SHADOWS
              "lime",        # 4 - VEGETATION
              "yellow",      # 5 - NOT_VEGETATED
              "blue",        # 6 - WATER
              "grey",        # 7 - UNCLASSIFIED
              "lightgrey",        # 8 - CLOUD_MEDIUM_PROB
              "darkgrey",        # 9 - CLOUD_HIGH_PROB
              "aqua",        # 10 - THIN_CIRRUS
              "magenta"]     # 11 - SNOW
              
bounds_scl = np.arange(0, 13, 1)

![SCL_legend](./img/SCL.png)

The figure above explains the value of each class a pixel can take within the SCL layer. This values are widely used for filtering out defective pixels, in our case study we can expect lots of pixels masked by clouds.

## 2.1 - Building the composite image dataset

The objective of this section its to build a composite-image (dataset) for 
the basin of the river previous to the storm. Then in the next section we will compute statistics in orde to asses the overflowed area

In [ ]:
scl = rioxarray.open_rasterio("./sentinel2/out/SCL/S2B2A_20200113_008_tordera_SCL_10.tif", masked=True)

scl_masked = scl.where(
                            (scl == 2) | # dark_area
                            (scl == 4) | # vegetation
                            (scl == 5) | # not_vegetated
                            (scl == 6)   # water
                          ) 

fig, axs = plt.subplots(2,1, figsize = (15,13))
scl.plot(ax = axs[0], colors = scl_colors, levels = bounds_scl)
axs[0].title.set_text("Scheme Classification Layer (SCL) - 2020-01-13")
plt.plot()

scl_masked.plot(ax = axs[1],  colors = scl_colors, levels = bounds_scl)
axs[1].title.set_text("SCL masked - 2020-01-13")
plt.show()

The `SCL masked-2020-01-13` image reveals the issue for the clouds. To build a complete composite of the basin we need to stack lots of images, filling the gaps.

In [ ]:
scl_dir = "./sentinel2/out/SCL/"

scl_files = [join(scl_dir, f) for f in listdir(scl_dir) if isfile(join(scl_dir, f))]
scl_files_dates = [S2_getDate(f).strftime("%Y-%m-%d") for f in scl_files]
scl_files_xarray = [rioxarray.open_rasterio(f) for f in scl_files] 

scl_dict = dict(zip(scl_files_dates, scl_files_xarray))

In [ ]:
scl_ds = xr.Dataset(scl_dict)
scl_ds.data_vars

In [ ]:
scl_ds_masked = scl_ds.where(
                    (scl_ds == 2) | # dark_area
                    (scl_ds == 4) | # vegetation
                    (scl_ds == 5) | # not_vegetated
                    (scl_ds == 6)   # water
                )

scl_ds_masked

In [ ]:
reduced = scl_ds_masked.reduce(scl_ds_masked['2020-01-08'].combine_first) 
reduced = reduced.reindex(y = reduced.y[::-1])
reduced = reduced.assign_coords(x = scl_ds_masked.x,
                                y = scl_ds_masked.y)

Now in `reduced` we have a composite image with almost all defective areas filled with the actual values of the land.

<div class="alert alert-block alert-info">
<b>Note:</b> Real composites are built using months worth of satellite images, in this process all the small gaps are filled. Here for the sake of the example we will omit this step.
</div>

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (15,7))
reduced['2020-01-13'].plot(ax = axs, colors = scl_colors, levels = bounds_scl)
axs.title.set_text("Scheme Classification Layer (SCL)")
plt.show()

## 2.2 - The post overlfow image 

In [ ]:
! ls -l ./sentinel2/out/SCL

In [ ]:
post_overflow_files = [
   "./sentinel2/out/SCL/S2A2A_20200128_008_tordera_SCL_10.tif",
   "./sentinel2/out/SCL/S2B2A_20200202_008_tordera_SCL_10.tif",
   "./sentinel2/out/SCL/S2A2A_20200207_008_tordera_SCL_10.tif",
   "./sentinel2/out/SCL/S2B2A_20200212_008_tordera_SCL_10.tif",
]

ncol = 2
nrow = len(post_overflow_files)//2

fig, axes = plt.subplots(nrow, ncol, figsize=(20,4*nrow),
                         sharex = True,
                         sharey = True,
                         constrained_layout = False)

for idx, ax in enumerate(np.ravel(axes)) : 
    try : 
        scl = rioxarray.open_rasterio(post_overflow_files[idx])
        scl.plot(ax=ax, colors = scl_colors, levels = bounds_scl)
        ax.title.set_text("SCL masked - %s" % S2_getDate(post_overflow_files[idx]) )

        
    except Exception as e :
        print("An error occurred while reading data for file %s " % post_overflow_files[idx])
        print("Original message: %s " % e)
    

In [ ]:
overflow = rioxarray.open_rasterio("./sentinel2/out/SCL/S2B2A_20200202_008_tordera_SCL_10.tif", masked=True)

overflow_masked = overflow.where(
                            (scl == 2) | # dark_area
                            (overflow == 4) | # vegetation
                            (overflow == 5) | # not_vegetated
                            (overflow == 6)   # water
                          ) 

fig, axs = plt.subplots(2,1, figsize = (15,13))
overflow.plot(ax = axs[0], colors = scl_colors, levels = bounds_scl)
axs[0].title.set_text("SCL 2022-02-02 - POST OVERFLOW")

overflow_masked.plot(ax = axs[1],  colors = scl_colors, levels = bounds_scl)
axs[1].title.set_text("SCL 2022-02-02 - POST OVERFLOW")

plt.show()

# 3 - Computing the overflow

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (15,7))
diff = reduced['2020-01-23'] - overflow_masked
diff.plot()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (15,7))
diff = reduced['2020-01-13'] - overflow
diff.plot()
plt.show()

## 3.1 -  Overflowed area

## 3.1 - Affected population